<a href="https://colab.research.google.com/github/laddhashreya2000/AI-Email-Classifier/blob/main/models/Data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setting up Root Dir

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# change this path to the cloned git repository
%cd /content/drive/My Drive/Project_Softie


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/.shortcut-targets-by-id/1aiL_ODB7FUeFLuLx8a2ZnWgmpA43A_uw/Project_Softie


#Importing Libraries

In [ ]:
import numpy as np
import pandas as pd

# For regular expressions
import re
# For handling string
import string
# For performing mathematical operations
import math

In [ ]:
data = pd.read_csv('./files/train_3.csv', encoding='mac_roman')

In [ ]:
data.head(10)

,Unnamed: 0,content,label
0,0,FW: Request for Information - ABC XYZ \r\n \r\...,retirements
1,1,FW: Test Pension Scheme \r\n \r\nFrom: ABC XYZ...,mdu
2,2,Change of Address - TEST SCHEME\r\nFrom: test....,mdu
3,3,FW: URGENT: ABC XYZ - Test Pension Fund ...,transfers
4,4,FW: ZZ123123Z \r\n \r\nFrom: ABC XYZ <abc.xyz@...,transfers
5,5,Letter of Authority - ABC\r\nFrom: test.ifa@fi...,transfers
6,6,Change of address\r\nFrom: abc.xyz@testmail.co...,mdu
7,7,FW: RBC Retirement Option Form \r\n \r\nFrom: ...,retirements
8,8,DEF Pension D.O.B 01/01/1400 National Insuranc...,mdu
9,9,FW: Test Pensions Pension Scheme: ref TST12345...,retirements


#Basic Data Preprocesing

In [ ]:
data1 = data

data1["text"]=data1["content"]

#removing null spaces
print(data1["content"].isnull().sum())
print(data1["label"].isnull().sum())

data1 = data1.dropna(axis=0, subset=['content']).reset_index(drop=True)


0
0


In [ ]:
len(data1)

297

In [ ]:
#website, URLs removal
pattern2 = r"(https?:\/\/) (\s)*(www\.)?(\s)* ((\w|\s)+\.)*([\w\-\s]+\/)*([\w-]+) ((\?) ?[\w\s]*=\s+[\w\%&]*)*"
pattern1 = r"((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*"

data1['text'] = data1['text'].str.replace(pattern1, '')

data1['text'] = data1['text'].str.replace(pattern2, '')


In [ ]:
data1.head(5)

,Unnamed: 0,content,label,text
0,0,FW: Request for Information - ABC XYZ \r\n \r\...,retirements,FW: Request for Information - ABC XYZ \r\n \r\...
1,1,FW: Test Pension Scheme \r\n \r\nFrom: ABC XYZ...,mdu,FW: Test Pension Scheme \r\n \r\nFrom: ABC XYZ...
2,2,Change of Address - TEST SCHEME\r\nFrom: test....,mdu,Change of Address - TEST SCHEME\r\nFrom: <>\r\...
3,3,FW: URGENT: ABC XYZ - Test Pension Fund ...,transfers,FW: URGENT: ABC XYZ - Test Pension Fund ...
4,4,FW: ZZ123123Z \r\n \r\nFrom: ABC XYZ <abc.xyz@...,transfers,FW: ZZ123123Z \r\n \r\nFrom: ABC XYZ <>\r\nSen...


In [ ]:
#removing html tags
html = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')

data1['text'] = data1['text'].str.replace(html, ' ')


In [ ]:
# cleaning email
email = re.compile('\S*@\S*\s')
data1['text'] = data1['text'].str.replace(email, '')


In [ ]:
#cleaning tags
tags = re.compile('@\S*\s')
data1['text'] = data1['text'].str.replace(r'@\S*\s', '')


In [ ]:
#removing punctuations
data1['text'] = data1['text'].str.replace(r'[^\w\s]', ' ')


In [ ]:
#removing non ascii characters
data1['text'] = data1['text'].astype("string", copy = False)

data1['text'] = data1['text'].apply(lambda x: x.encode('ascii','ignore').decode())


In [ ]:
#removing numerical data
data1['text'] = data1['text'].str.replace(r'\d+', '')


In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
nltk.download('wordnet')

#removing stopwords
new_stp = ["FW:", "fw", "RE:", "re", "URGENT:", "urgent", "abc", "xyz", "from", "to", "subject", "test", "i", "sent", "to"]
stp = stopwords.words('english')
stp.extend(new_stp)

pattern = re.compile(r'\b(' + r'|'.join(stp) + r')\b\s*')
data1['text'] = data1['text'].str.replace(pattern, '')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
print(stp)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
#Converting Dataset into lower-case
data1['text'] = data1['text'].str.lower()

In [ ]:
#removing extra space
data1['text'] = data1['text'].str.replace(r'\s+', ' ')

In [ ]:
data1.drop(["content"], axis=1, inplace=True)
data1.head(5)

,Unnamed: 0,label,text
0,0,retirements,fw request information abc xyz from abc xyz se...
1,1,mdu,fw test pension scheme from abc xyz sent frida...
2,2,mdu,change address test scheme from sent november ...
3,3,transfers,fw urgent abc xyz test pension fund our ref ur...
4,4,transfers,fw zzz from abc xyz sent friday november pm to...


In [ ]:
# Lemmatization
import spacy
nlp = spacy.load("en_core_web_sm")

# keep_pos = ['PROPN', 'NOUN']
data1['t1'] = ''
i=0
for text in data1['text']:
    data1['t1'][i] = ' '.join([str(tok.lemma_) for tok in nlp(str(text))])
    i+=1
# self.df = self.df[self.df.astype(str)['nouns'] != '[]']
# self.df = self.df.reset_index(drop=True)
        
# l1 = data1['text']
# l2 = []

# lem = nltk.stem.wordnet.WordNetLemmatizer()
# ps = nltk.stem.porter.PorterStemmer()
        
# for t in l1:
#    #lst_txt = [lem.lemmatize(word) for word in t]
#    lst_txt = [ps.stem(word) for word in t]
#    print(lst_txt)
#    listToStr = ' '.join([str(elem) for elem in lst_txt]) 
#    l2.append(listToStr)


# data1['text'] = l2;

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
# finding frequencies of different words 
key_words={}

for i in data1['text']:
  arr = i.split(' ')
  for word in arr:
    if word in key_words.keys():
      key_words[word]+=1
    else:
      key_words[word]=1


for i in key_words.keys():
  if key_words[i]<3:
    stp.append(i)

sort_orders = sorted(key_words.items(), key=lambda x: x[1], reverse=True)   

for j in sort_orders:
  print(j)

('test', 1931)
('pension', 1310)
('abc', 1194)
('please', 1029)
('i', 1012)
('xyz', 828)
('to', 721)
('sent', 668)
('subject', 662)
('from', 647)
('scheme', 607)
('pensions', 486)
('november', 485)
('address', 410)
('email', 391)
('regards', 348)
('information', 321)
('transfer', 317)
('retirement', 313)
('', 305)
('us', 288)
('plan', 286)
('fund', 265)
('thank', 263)
('if', 262)
('tst', 261)
('dear', 254)
('aa', 253)
('date', 251)
('road', 248)
('pm', 243)
('member', 238)
('would', 237)
('street', 237)
('number', 234)
('contact', 231)
('could', 223)
('provide', 218)
('request', 213)
('support', 213)
('zzz', 207)
('kind', 202)
('re', 201)
('centre', 195)
('benefits', 193)
('fw', 190)
('customer', 190)
('mr', 186)
('details', 184)
('friday', 183)
('confirm', 183)
('monday', 178)
('change', 168)
('e', 167)
('def', 164)
('value', 163)
('good', 160)
('fin', 157)
('mail', 156)
('letter', 153)
('yours', 151)
('regarding', 150)
('sincerely', 147)
('ifa', 145)
('thanks', 142)
('corp', 141)
('h

In [ ]:
data1

,Unnamed: 0,label,text,t1
0,0,retirements,request information monday november pm pension...,request information monday november pm pension...
1,1,mdu,pension scheme friday november pm rbc pension ...,pension scheme friday november pm rbc pension ...
2,2,mdu,change address scheme november change address ...,change address scheme november change address ...
3,3,transfers,pension fund ref friday november pm pension fu...,pension fund ref friday november pm pension fu...
4,4,transfers,zzz friday november pm pension fund zzz dear s...,zzz friday november pm pension fund zzz dear s...
...,...,...,...,...
292,292,mdu,password reset fin corp support monday novembe...,password reset fin corp support monday novembe...
293,293,mdu,member address change zzz november member addr...,member address change zzz november member addr...
294,294,mdu,change address original message november chang...,change address original message november chang...
295,295,transfers,guaranteed transfer value zzz november cc guar...,guaranteed transfer value zzz november cc guar...


In [ ]:
data1.to_csv('./files/dataset_processed.csv')